<a href="https://colab.research.google.com/github/ChefHorst/fsc-sar/blob/main/FSC_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tri-Band Logging Visualizaion using Palsar-2



## Imports and Map Initialization

In [ ]:
import ee

import ipywidgets as widgets

from google.colab import drive

# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='fsc-demo-6775')

Map = geemap.Map(basemap='HYBRID')

Mounted at /content/drive


## Map Widgets and Buttons

In [ ]:
start_btn = widgets.Button(description = "Start")

year1 = widgets.IntText(
    value=2015,
    description='Year 1:',
    disabled=False
)

year2 = widgets.IntText(
    value=2017,
    description='Year 2:',
    disabled=False
)

year3 = widgets.IntText(
    value=2020,
    description='Year 3:',
    disabled=False
)

ui = widgets.VBox((year1,year2,year3, start_btn))


## Functions

In [ ]:
# Masks out edge pixels in the image that have a value less than -30.0 dB
def mask_edge(image):
  # Create a mask for pixels with values less than -30 dB
  edge = image.lt(-30.0)
  # Combine the original mask with the inverse of the edge mask
  masked_image = image.mask().And(edge.Not())
  # Apply the updated mask to the image
  return image.updateMask(masked_image)

# Converts HH-polarized SAR image data (DN) to decibels
def to_decibel(image):
  """Converts PALSAR-2 ScanSAR HH polarization from DN to decibel (dB)."""
  # γ0 (in dB) = 10 * log10(DN^2) - 83
  gamma0_db = image.pow(2).log10().multiply(10).subtract(83.0)
  # Copy timestamp metadata from original image
  return gamma0_db.copyProperties(image, ['system:time_start', 'system:time_end'])

# Creates a multi-band image from mean summer composites over a list of years
def createImg(geom, years):

  # Clips image to the specified geometry (default is geom passed to the function)
  def clip_bounds(image, shape=geom):
    return image.clip(geom)

  # Load and preprocess the ALOS PALSAR-2 HH image collection
  img_hh = (ee.ImageCollection('JAXA/ALOS/PALSAR-2/Level2_2/ScanSAR')
      .select('HH')                      # Select HH polarization band
      .filter(ee.Filter.bounds(geom))   # Filter by spatial bounds
      .map(to_decibel)                  # Convert to decibel scale
      .map(mask_edge)                   # Mask out noisy edge pixels
      .map(clip_bounds)                 # Clip to the input geometry
    )

  tri_band = []  # List to store seasonal mean images for each year

  for yr in years:
    year = yr.value
    # Compute the summer mean image for the year and rename it as a band (e.g., 'B2020')
    tri_band.append(
      img_hh.filter(ee.Filter.date(f'{year}-03-01', f'{year}-09-20'))
            .mean()
            .rename('B' + str(year))
    )

  # Concatenate all yearly mean images into a multi-band image
  tri_band_image = ee.Image.cat(tri_band)

  # Also prepare a median image from a longer summer season (2020–2021)
  hh_summer = img_hh.filter(ee.Filter.date('2020-03-01', '2021-11-20')).median()

  # Return only the multi-band image (hh_summer is computed but not returned)
  return tri_band_image

## Creating the Map

In [ ]:
# Clear any existing layers, drawings, or settings from the map
Map.clear()

# Re-initialize the map with a hybrid satellite-street basemap
Map = geemap.Map(basemap='HYBRID')

# Define a callback function that runs when the 'start' button is clicked
def on_click_start(b):
  # Proceed only if a region of interest (ROI) was manually selected by the user
  if Map.user_roi is not None:

    # Check if the user has drawn any features (e.g., a polygon)
    if len(Map.draw_features) > 0:
      # Use predefined years (e.g., year1, year2, year3) for creating image bands
      years = [year1, year2, year3]

      # Get the geometry of the first drawn feature
      geom = Map.draw_features[0].geometry()

      # Clear the drawing control so the map is clean
      Map.draw_control.clear()

      # Generate and add a multi-band image layer to the map using the selected geometry and years
      Map.add_layer(
        createImg(geom, years),
        {'min': -30, 'max': 0, 'gamma': 0.8},  # Visualization parameters for SAR decibel image
        "HH"  # Name of the layer
      )

# Attach the `on_click_start` function to the button's click event
start_btn.on_click(on_click_start)

# Add a custom user interface widget (`ui`) to the map in the bottom right corner
Map.add_widget(ui, 'bottomright')

# Display the map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…